# Repaso Unidad II: Series Temporales

**Curso:** Forecasting con Python  
**Unidad:** Métodos autoregresivos y automatizados  
**Notebook:** `repaso_unidad2_documentado.ipynb`

## Objetivo

Construir un flujo completo de *forecasting univariante* en español, replicable de principio a fin, con énfasis en modelos autoregresivos y automatizados inspirados en el capítulo 4 de Lazzeri (2020).  
Se compararán dos enfoques:

- **AutoReg (AR)** para modelar dependencia con rezagos.
- **SARIMAX** para capturar estructura autoregresiva, integración y componente estacional.

## Introducción y contexto

### ¿Qué es forecasting univariante?
Es la predicción de valores futuros de una única variable ordenada en el tiempo (por ejemplo, ventas mensuales), usando su propio historial.

### ¿Cuándo usar AR, ARIMA y SARIMAX?
- **AR (AutoReg):** útil cuando la serie es aproximadamente estacionaria y los rezagos explican bien la dinámica.
- **ARIMA:** apropiado cuando existe tendencia/no estacionariedad que puede corregirse con diferenciación.
- **SARIMAX:** recomendable cuando además hay estacionalidad explícita (mensual, trimestral, etc.) y se desea modelarla de forma estructurada.

## Librerías

**Propósito del bloque:** importar librerías necesarias y fijar semilla para reproducibilidad.  
**Qué se espera observar:** entorno listo para análisis, modelado y evaluación.  
**Cómo interpretar:** si este bloque corre sin errores, el notebook puede ejecutarse en orden (*Run All*).

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.statespace.sarimax import SARIMAX

from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

np.random.seed(42)
plt.style.use('default')

## Carga y preparación de datos

**Propósito del bloque:** generar una serie sintética realista (sin datos privados) con tendencia, estacionalidad y ruido.  
**Qué se espera observar:** una serie mensual con patrón temporal plausible para forecasting.  
**Cómo interpretar:** esta serie será la base para EDA, entrenamiento y evaluación comparativa.

In [ ]:
# Serie mensual de 8 años
n_periodos = 8 * 12
fechas = pd.date_range(start='2015-01-01', periods=n_periodos, freq='MS')

tendencia = np.linspace(50, 120, n_periodos)
estacionalidad = 12 * np.sin(2 * np.pi * np.arange(n_periodos) / 12)
ruido = np.random.normal(loc=0, scale=4, size=n_periodos)

valores = tendencia + estacionalidad + ruido
serie = pd.Series(valores, index=fechas, name='demanda')

df = serie.to_frame()

**Interpretación breve:** la serie incluye componentes típicos de negocios (crecimiento, ciclos estacionales y variación aleatoria), lo que la hace adecuada para practicar modelos AR y SARIMAX.

## Inspección inicial

**Propósito del bloque:** validar estructura del dataset con `head()`, `shape` e información general.  
**Qué se espera observar:** una sola columna numérica y un índice temporal mensual.  
**Cómo interpretar:** confirma que los datos están listos para análisis univariante.

In [ ]:
df.head(), df.shape

In [ ]:
df.info()

**Interpretación breve:** se dispone de una serie univariante continua en frecuencia mensual, sin necesidad de transformación adicional para iniciar el análisis exploratorio.

## EDA de serie temporal

**Propósito del bloque:** visualizar la evolución temporal e identificar señales de tendencia/estacionalidad.  
**Qué se espera observar:** crecimiento progresivo y oscilaciones periódicas.  
**Cómo interpretar:** estos patrones justifican comparar un modelo AR con otro que maneje estacionalidad (SARIMAX).

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(df.index, df['demanda'], color='tab:blue', linewidth=1.8)
plt.title('Serie temporal sintética: demanda mensual')
plt.xlabel('Fecha')
plt.ylabel('Nivel de demanda')
plt.grid(alpha=0.3)
plt.show()

### Lag plot, ACF y PACF

**Propósito del bloque:** evaluar dependencia temporal y estructura de autocorrelación.  
**Qué se espera observar:** relación positiva en rezagos cercanos y picos estacionales en ACF/PACF.  
**Cómo interpretar:** ayuda a seleccionar órdenes iniciales para modelos autoregresivos.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5, 5))
lag_plot(df['demanda'], lag=1, ax=ax)
ax.set_title('Lag plot (rezago 1)')
plt.show()

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
plot_acf(df['demanda'], lags=36, ax=axes[0])
plot_pacf(df['demanda'], lags=36, ax=axes[1], method='ywm')
axes[0].set_title('ACF')
axes[1].set_title('PACF')
plt.tight_layout()
plt.show()

**Interpretación breve:** la dependencia temporal es clara y aparecen patrones periódicos, por lo que tiene sentido evaluar tanto AR como SARIMAX estacional.

## Split temporal (train/test)

**Propósito del bloque:** separar entrenamiento y prueba de forma cronológica (80/20), sin *shuffle*.  
**Qué se espera observar:** train con observaciones iniciales y test con observaciones finales.  
**Cómo interpretar:** este esquema simula un escenario real de predicción futura.

In [ ]:
train_size = int(len(df) * 0.8)
train = df.iloc[:train_size].copy()
test = df.iloc[train_size:].copy()

print(f'Tamaño train: {train.shape}')
print(f'Tamaño test: {test.shape}')
print(f'Rango train: {train.index.min().date()} -> {train.index.max().date()}')
print(f'Rango test:  {test.index.min().date()} -> {test.index.max().date()}')

**Interpretación breve:** la partición respeta el orden temporal y evita fuga de información del futuro al pasado.

## Modelo AR (AutoReg)

**Propósito del bloque:** entrenar un modelo autoregresivo con rezagos razonables (12 meses) y generar pronóstico en test.  
**Qué se espera observar:** predicciones que siguen la dinámica general de la serie.  
**Cómo interpretar:** un MAPE más bajo indica mejor capacidad predictiva en el horizonte evaluado.

In [ ]:
ar_lags = 12
modelo_ar = AutoReg(train['demanda'], lags=ar_lags, old_names=False)
resultado_ar = modelo_ar.fit()

pred_ar = resultado_ar.predict(start=test.index[0], end=test.index[-1])
pred_ar = pred_ar.reindex(test.index)

assert len(pred_ar) == len(test), 'Longitudes incompatibles entre predicción AR y test.'

mape_ar = mean_absolute_percentage_error(test['demanda'], pred_ar) * 100
mae_ar = mean_absolute_error(test['demanda'], pred_ar)
rmse_ar = mean_squared_error(test['demanda'], pred_ar, squared=False)

print(f'MAPE AR: {mape_ar:.2f}%')
print(f'MAE AR: {mae_ar:.2f}')
print(f'RMSE AR: {rmse_ar:.2f}')

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(train.index, train['demanda'], label='Train', color='tab:blue', alpha=0.7)
plt.plot(test.index, test['demanda'], label='Real (Test)', color='black', linewidth=2)
plt.plot(test.index, pred_ar, label='Predicción AR', color='tab:orange', linestyle='--', linewidth=2)
plt.title('AutoReg: real vs predicción')
plt.xlabel('Fecha')
plt.ylabel('Demanda')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

**Interpretación breve:** el modelo AR captura parte de la inercia temporal; sin embargo, su desempeño puede verse limitado cuando la estacionalidad es marcada.

## Modelo SARIMAX

**Propósito del bloque:** entrenar un modelo SARIMAX con estacionalidad mensual para capturar mejor la estructura periódica.  
**Qué se espera observar:** ajuste más fino en ciclos estacionales frente al modelo AR.  
**Cómo interpretar:** comparar su MAPE contra AR permite decidir cuál generaliza mejor en test.

In [ ]:
modelo_sarimax = SARIMAX(
    train['demanda'],
    order=(1, 1, 1),
    seasonal_order=(1, 1, 1, 12),
    enforce_stationarity=False,
    enforce_invertibility=False
)
resultado_sarimax = modelo_sarimax.fit(disp=False)

pred_sarimax = resultado_sarimax.predict(start=test.index[0], end=test.index[-1])
pred_sarimax = pred_sarimax.reindex(test.index)

assert len(pred_sarimax) == len(test), 'Longitudes incompatibles entre predicción SARIMAX y test.'

mape_sarimax = mean_absolute_percentage_error(test['demanda'], pred_sarimax) * 100
mae_sarimax = mean_absolute_error(test['demanda'], pred_sarimax)
rmse_sarimax = mean_squared_error(test['demanda'], pred_sarimax, squared=False)

print(f'MAPE SARIMAX: {mape_sarimax:.2f}%')
print(f'MAE SARIMAX: {mae_sarimax:.2f}')
print(f'RMSE SARIMAX: {rmse_sarimax:.2f}')

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(train.index, train['demanda'], label='Train', color='tab:blue', alpha=0.7)
plt.plot(test.index, test['demanda'], label='Real (Test)', color='black', linewidth=2)
plt.plot(test.index, pred_sarimax, label='Predicción SARIMAX', color='tab:green', linestyle='--', linewidth=2)
plt.title('SARIMAX: real vs predicción')
plt.xlabel('Fecha')
plt.ylabel('Demanda')
plt.legend()
plt.grid(alpha=0.3)
plt.show()

**Interpretación breve:** al modelar explícitamente la estacionalidad, SARIMAX suele mejorar la estabilidad del pronóstico en series mensuales cíclicas.

## Evaluación comparativa

**Propósito del bloque:** resumir métricas clave por modelo y seleccionar el mejor según MAPE (menor es mejor).  
**Qué se espera observar:** una tabla comparativa clara y una identificación automática del mejor modelo.  
**Cómo interpretar:** esta síntesis facilita la toma de decisión para despliegue o iteración.

In [ ]:
comparacion = pd.DataFrame({
    'Modelo': ['AutoReg', 'SARIMAX'],
    'MAPE (%)': [mape_ar, mape_sarimax]
}).sort_values('MAPE (%)', ascending=True).reset_index(drop=True)

mejor_modelo = comparacion.loc[0, 'Modelo']
mejor_mape = comparacion.loc[0, 'MAPE (%)']

comparacion

In [ ]:
print(f'Mejor modelo por MAPE: {mejor_modelo} ({mejor_mape:.2f}%)')

**Interpretación breve:** el modelo con menor MAPE se considera más preciso en este conjunto de prueba. La elección final debe validarse también con estabilidad temporal y contexto de negocio.

## Conclusiones

- El *forecasting univariante* permite construir líneas base sólidas usando solo la historia de la variable objetivo.
- **AutoReg** ofrece una implementación rápida y útil para dependencia de corto/mediano plazo.
- **SARIMAX** resulta especialmente conveniente cuando hay estacionalidad observable, pudiendo reducir el error relativo.
- La comparación mediante **MAPE (%)** facilita la selección del modelo ganador de forma interpretable.
- **Limitaciones:** serie sintética, un solo esquema de partición, y búsqueda de hiperparámetros no exhaustiva.
- **Mejoras posibles:** validación *walk-forward*, ajuste sistemático de órdenes (grid/random search), incorporación de variables exógenas y diagnóstico residual más profundo.

## Referencias

Lazzeri, F. (2020). *Machine Learning for Time Series Forecasting with Python*. Wiley.